In [0]:
%pip install --upgrade langchain faiss-cpu mlflow

  Using cached langchain-0.2.15-py3-none-any.whl (1.0 MB)
  Using cached faiss_cpu-1.8.0.post1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (27.0 MB)
  Using cached mlflow-2.16.0-py3-none-any.whl (26.6 MB)
  Using cached aiohttp-3.10.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.2 MB)
  Using cached langchain_text_splitters-0.2.2-py3-none-any.whl (25 kB)
  Using cached PyYAML-6.0.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (751 kB)
  Using cached SQLAlchemy-2.0.32-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)
  Using cached async_timeout-4.0.3-py3-none-any.whl (5.7 kB)
  Using cached langsmith-0.1.108-py3-none-any.whl (150 kB)
  Using cached langchain_core-0.2.37-py3-none-any.whl (396 kB)
  Using cached alembic-1.13.2-py3-none-any.whl (232 kB)
  Using cached docker-7.1.0-py3-none-any.whl (147 kB)
  Using cached Markdown-3.7-py3-none-any.whl (106 kB)
  Using cached mlflow_skinny-2.16.0-py3-none-any.whl (5.6 MB)
  Usi

In [0]:
%pip install -U langchain-openai langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.5/776.5 kB 18.8 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
%pip install -U langchain-openai

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
%pip install faiss-cpu

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
dbutils.library.restartPython()

In [0]:

import sys
# sys.path.append('/usr/lib/python3.9/site-packages')


from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import FAISS

from langchain_core.runnables import RunnableLambda
from langchain.retrievers.multi_query import MultiQueryRetriever


import os
os.environ["OPENAI_API_KEY"]="이곳에 api key를 넣어주세요"


## meta chain, reveiw chain 

class Retriever:
    def __init__(self, retriever_type):
        self.retriever_type = retriever_type

        # Initialize common attributes first
        self.embed_model = OpenAIEmbeddings()  # Embedding model initialization
        self.llm_model = ChatOpenAI(model_name="gpt-4o-mini")  # LLM model initialization
        
        
        # Set vector store path based on the retriever type
        if retriever_type == "review":
            self.vector_store_path=os.path.abspath("/home/ec2-user/workspace/workspace/vectorDB/review2")
            # Initialize the database (FAISS) before using it in retrievers
            self.db = FAISS.load_local(self.vector_store_path, self.embed_model, allow_dangerous_deserialization=True)

            self.retriever = MultiQueryRetriever.from_llm(
                retriever=self.db.as_retriever(search_type="mmr"),
                llm=self.llm_model
           )
            self.prompt_template=""""
            You are an expert in 'review information' \
            You will recommend places to the user based on their questions. When recommending places, it will do so based on Google Maps review data."
            This review data contains information about various places.
            Explaining columns in the data, 

            gmap_id: Unique map ID for your store.
            rating: the rating of a place on a scale of 1 to 5.
            review : reviews for the place. The review content includes information related to 'service(great customer service, great service, good service, friendly staff)', 'food taste(great food, good food, delicious food)', 'experience(great experience, great place, nice place, good place, here again next time, back again next time, back here next time)', 'tips for the place', 'time(Date the review was uploaded)', 'waiting time', 'great place to go with someone(friend, family)'

            response_of_store_owner : place owner's response to user reviews.
            time : Date the review was uploaded (review date, year).
    
            \

            Always answer questions starting with "Accroding to review information..."
            For 'review summary', please summarize the review contents mainly in relation to the questions\
            Respond to the following question:{context}

            Question: {question}
            Answer:
            \nrecommended place: ** name **
            \n
            \n- review summary:
            \n- [gmap_id]
            
            """


        elif retriever_type == "meta":
            self.vector_store_path=os.path.abspath("/home/ec2-user/workspace/workspace/vectorDB/meta2")
            # Initialize the database (FAISS) before using it in retrievers
            self.db = FAISS.load_local(self.vector_store_path, self.embed_model, allow_dangerous_deserialization=True)
            self.retriever = MultiQueryRetriever.from_llm(
                retriever=self.db.as_retriever(),
                llm=self.llm_model
            )
             # self.retriever = self.db.as_retriever(search_type="mmr", search_kwargs={"k": 2})
            self.prompt_template=""""
            You are an expert in 'meta information'. \
            You will recommend places to the user based on their questions. When recommending places, it will do so based on Google Maps metadata."
            This meta data contains information about various places.
            Explaining columns in the data, 

            name : place name 
            category_filtered: The category of the place such as restaurants, retail shops, park, beach, etc. 
            MISC_filtered: Various other information about the store such as Accessibility, Activities, Amenities, Atmosphere, Crowd, Dining_options, From_the_business, Getting_here,Health_and_safety,Highlights,Lodging_options,Offerings,Payments,Planning,Popular_for ,Recycling ,Service_options
            address: Address of the place 
            avg_rating: the average rating of a place on a scale of 1 to 5.
            description: A brief description of the place
            gmap_id: Unique map ID for your store
            hours_filtered: place operating hours, business hours (The operating days and times) 
            latitude: Latitude of place location
            longitude: longitude of place location
            num_of_reviews: Number of google map reviews for the place
            price: Prices for items sold in a store. 
            state: Current operating status
            url: url link to the place
            relative_results: gmap_ids of places similar to the given place
    
            \

            Always answer questions starting with "Accroding to meta information...".\
            Respond to the following question:{context}

            Question: {question}
            Answer:
            \nrecommended place: ** name **
            \n
            \n- Address: 
            \n- avg_rating:
            \n- Number of Reviews:
            \n- hours:
            \n- category:
            \n- [Google Maps Link]
            \n- [gmap_id]

            \n\nPlease provide only one recommendation by returning the 1 most relevant result.
            """

        else:
            raise ValueError("Invalid retriever type. Choose 'review' or 'meta'.")


        # Set up the prompt template
        self.prompt_template = ChatPromptTemplate.from_template(self.prompt_template)

    def format_docs(self, docs):
        return '\n\n'.join([d.page_content for d in docs])

    # 번역을 위한 LLM 호출 함수
    def translate_with_llm(self, text, target_language):
        # 번역을 위해 OpenAI 모델을 사용하여 LLM을 호출하는 프롬프트 설정
        translation_prompt = f"Translate the following text to {target_language}:\n\n{text}"
        translated_text = self.llm_model.invoke(translation_prompt)
        return translated_text

    # 한국어 -> 영어 번역
    def translate_to_english(self, text):
        return self.translate_with_llm(text, "English")

    # 영어 -> 한국어 번역
    def translate_to_korean(self, text):
        return self.translate_with_llm(text, "Korean")

    def get_chain(self):
        chain = (
            {'context': lambda x: self.format_docs(self.retriever.invoke(self.translate_to_english(x["question"]))),
             'question': lambda x: x["question"]}
            | self.prompt_template
            | self.llm_model
        )
        return chain
    

    def search(self, query):
        chain = self.get_chain()
        res = chain.invoke({"question": query})
        return res.content



# Meta 정보와 리뷰 정보를 처리하는 체인을 리턴하는 클래스
class KoreanChainClassifier:
    def __init__(self):
        # 모델 이름을 전달하여 초기화
        self.llm_model = ChatOpenAI(model_name="gpt-4o-mini")
        self.meta_retriever = Retriever("meta")
        self.review_retriever = Retriever("review")

    # 분류 체인 정의
    def get_classify_chain(self):
        # 템플릿 정의
        prompt = ChatPromptTemplate.from_template(
            """Classify the given user question as either `meta information`, `review information`. Please do not respond with more than one word.
            For reference, 
            'meta information' includes 'place name', 'place category', 'place rating', 'place address', 'place operating hours', 'price', 'place status', 'place MISC information(Accessibility, Activities, Amenities,Payments)', etc.  

            ‘review information’ includes 'service(great customer service, great service, good service, friendly staff)', 'food taste(great food, good food, delicious food)', 'experience(great experience, great place, nice place, good place, here again next time, back again next time, back here next time)', 'tips for the place', 'time(Date the review was uploaded)', 'cleanliness', 'waiting time', 'great place to go with someone(friend, family)' etc. 
            <question>
            {question}
            </question>

            Classification:"""
        )

        # 체인 생성
        cl_chain = (
            prompt
            | self.llm_model
            | StrOutputParser()
        )

        return cl_chain

    # 정보 분류에 따라 체인 라우팅
    def route(self, info):
        # 메타 정보가 포함된 경우 메타 체인 반환
        if "meta information" in info["topic"].lower():
            return self.meta_retriever.get_chain()
        # 리뷰 정보가 포함된 경우 리뷰 체인 반환
        elif "review information" in info["topic"].lower():
            return self.review_retriever.get_chain()
        # 그 외의 경우 메타 체인 반환
        else:
            return self.meta_retriever.get_chain()

    # 번역을 위한 LLM 호출 함수
    def translate_with_llm(self, text, target_language):
        # 번역을 위해 OpenAI 모델을 사용하여 LLM을 호출하는 프롬프트 설정
        translation_prompt = f"Translate the following text to {target_language}:\n\n{text}"
        translated_text = self.llm_model.invoke(translation_prompt)
        return translated_text

    # 한국어 -> 영어 번역
    def translate_to_english(self, text):
        return self.translate_with_llm(text, "English")

    # 영어 -> 한국어 번역
    def translate_to_korean(self, text):
        return self.translate_with_llm(text, "Korean")


    # 전체 체인 정의
    def get_full_chain(self):
        full_chain = (
            {"topic": self.get_classify_chain(), "question": lambda x: x["question"]}
            | RunnableLambda( 
                # 라우팅 함수를 전달
                self.route
            )
            | StrOutputParser()
        )

        return full_chain

    # 전체 체인을 실행하고 결과 반환
    def search(self, query):
        full_chain = self.get_full_chain()
        result = full_chain.invoke({"question": query})

        # 최종 결과를 한국어로 번역
        translated_result = self.translate_to_korean(result).content

        return translated_result


In [0]:
full_chain = KoreanChainClassifier()

In [0]:
print(full_chain.search("평점이 4.0 이상인 랍스터 레스토랑 추천해줘"))

메타 정보를 바탕으로, 평점이 4.0 이상인 추천 랍스터 레스토랑은 다음과 같습니다:

**추천 장소: 레드 롭스터**

- 주소: 1525 트래비스 블러바드, 페어필드, CA 94533
- 평균 평점: 4.2
- 리뷰 수: 1265
- 운영 시간: 월요일, 오전 11:30 - 오후 8:30 / 화요일, 오전 11:30 - 오후 8:30 / 수요일, 오전 11:30 - 오후 8:30 / 목요일, 오전 11:30 - 오후 8:30 / 금요일, 오전 11:30 - 오후 9:30 / 토요일, 오전 11:30 - 오후 9:30 / 일요일, 오전 11:30 - 오후 8:30
- 카테고리: 해산물 레스토랑, 미국식 레스토랑, 앵글러 피쉬 레스토랑, 바, 게 하우스, 이벤트 장소, 가족 레스토랑
- [구글 지도 링크](https://www.google.com/maps/place//data=!4m2!3m1!1s0x80851423d137e51b:0xd7a2d93bdc327e75?authuser=-1&hl=en&gl=us)
- [gmap_id] 0x80851423d137e51b:0xd7a2d93bdc327e75


In [0]:
print(full_chain.search("가족과 함께 가기 좋은 공원 추천해줘"))

다음 정보를 바탕으로 가족과 함께 즐거운 나들이를 위한 장소를 추천합니다:

**추천 장소: 베니 H. 포터 웨스트 아담스 애비뉴 기념 공원**

- 리뷰 요약: 이 공원은 가족 활동을 위한 넓은 공간과 좋은 개방감을 제공합니다. 다양한 연령대의 어린이에게 적합한 대형 놀이터, 피크닉 테이블, 농구장이 있습니다. 또한, 피트니스 공간과 반려동물용 개방 공간이 있어 가족 모임이나 외출에 적합한 다목적 장소입니다. 그러나 화장실과 같은 일부 편의시설은 더 자주 청소될 필요가 있습니다.

- [gmap_id: 0x80c2b8696ab728a7:0x2b4227d1397a4c4a]
